# Lesson 17 - Design Patterns

In [ ]:
#include <iostream>
#include <vector>
#include <fstream>
#include <string>
#include <memory>

## Parameters packs

In [ ]:
template<typename ...Ts>
void MyPPFunction(Ts... args) {
    std::cout << "I was passed " << sizeof...(args) << " arguments" << std::endl;
}

In [ ]:
MyPPFunction(1, 3, 4);
MyPPFunction("Hello world", "Hi there");

In [ ]:
template<typename ...Ts>
std::vector<std::common_type_t<Ts...>> make_vector(Ts... args) {
    return std::vector<std::common_type_t<Ts...>> {args...};
}

In [ ]:
auto my_vector {make_vector(1, 2, 3, 4, 5, 6)};
std::cout << "my_vector has " << my_vector.size() << " elements" << std::endl;

In [ ]:
template<typename T>
void PrintList(T arg) {
    std::cout << arg << std::endl;
}

template<typename T, typename ...Ts>
void PrintList(T arg, Ts... args) {
    std::cout << arg << ", ";
    PrintList(args...);
}

In [ ]:
PrintList(1, 2, 3, "hello world", 5.5, 7, "end");

## Initialiser lists

In [ ]:
template<typename T>
void MyInitFunction(std::initializer_list<T> args) {
    for (const auto& arg: args) {
        std::cout << "Argument: " << arg << std::endl;
    }
}

In [ ]:
MyInitFunction({1, 2, 3, 4, 5});
MyInitFunction({"Hi", "there"});

In [ ]:
MyInitFunction({1, 2, 3, 4, "Hi"}); // Error - not the same types

In [ ]:
// Note - no template now
void MyInitIntFunction(std::initializer_list<int> args) {
    for (const auto& arg: args) {
        std::cout << "Argument: " << arg << std::endl;
    }
}

In [ ]:
MyInitIntFunction({1, 2, 3, 4});

In [ ]:
MyInitIntFunction({1.2, 1.3}); // Error - not int

## Design patterns

This is a huge area in software development - there are [a lot of books on this topic](https://www.amazon.co.uk/s?k=design+patterns&ref=nb_sb_noss).

### Singleton

In [ ]:
struct Configuration {
    std::unique_ptr<std::ifstream> input_file;
    // Any other program-wide configuration options
};

In [ ]:
class ProgramConfiguration {
public:
    Configuration& GetConfiguration() {return configuration_;}
    static ProgramConfiguration& GetInstance() {
        static ProgramConfiguration instance; // Create an instance
        return instance; // Return the only instance
    }
private:
    ProgramConfiguration(); // Default constructor is private
    ProgramConfiguration(const ProgramConfiguration& other) = delete; // Do not allow the instance to be copied
    ProgramConfiguration(const ProgramConfiguration&& other) = delete; // Do not allow the instance to be moved
    ProgramConfiguration& operator=(const ProgramConfiguration &rhs) const = delete; //disallow copy-assignment operator
    ProgramConfiguration& operator=(const ProgramConfiguration &&rhs) const = delete; //disallow move-assignment operator
    ~ProgramConfiguration() {} // Private destructor controls who can destroy the object
    Configuration configuration_;
};

In [ ]:
// Deletages with a default filename
ProgramConfiguration::ProgramConfiguration() {
    std::string filename {"input.dat"}; // Load configuration from file or database
    
    // Use that to instantiate the configuration value in our (only) instance
    configuration_ = Configuration {std::make_unique<std::ifstream>(filename)};
} 

### Factory

In [ ]:
struct OptionData {
    // Option data struct for common data
};

In [ ]:
// Base class for all our options
class Option {
protected:
    OptionData data_; // Member for initial data
public:
    Option(const OptionData& initial_data) : data_ {initial_data} { }
    virtual double GetPayoff() const = 0;
};

In [ ]:
class EurCall : public Option {
public:
    EurCall(const OptionData& initial_data) : Option(initial_data) { }
    double GetPayoff() const { return 3; } // Should calculate!
};

In [ ]:
class EurPut : public Option {
public:
    EurPut(const OptionData& initial_data) : Option(initial_data) { }
    double GetPayoff() const { return 1; } // Should calculate!
};

In [ ]:
class AsianCall : public Option {
public:
    AsianCall(const OptionData& initial_data) : Option(initial_data) { }
    double GetPayoff() const { return 2; } // Should calculate!
};

In [ ]:
enum class OptionType {
    EurCall,
    EurPut,
    AsianCall,
    // etc.
};

In [ ]:
std::unique_ptr<Option> MyOptionFactory(const OptionType type, const OptionData& initial_data) {
    switch(type) {
        case OptionType::EurCall:
            return std::make_unique<EurCall>(initial_data);
        case OptionType::EurPut:
            return std::make_unique<EurPut>(initial_data);
        case OptionType::AsianCall:
            return std::make_unique<AsianCall>(initial_data);
    }
}

In [ ]:
std::unique_ptr<Option> my_call {MyOptionFactory(OptionType::EurCall, OptionData{})};

In [ ]:
std::cout << my_call->GetPayoff() << std::endl;

In [ ]:
my_call = MyOptionFactory(OptionType::AsianCall, OptionData{}); // Move assignment - valid

In [ ]:
std::cout << my_call->GetPayoff() << std::endl;